## Développement d’une solution de mapping de données de séquençage à haut-débit sur un génome de référence

#### Mise en place d'un algorithme permettant de chercher un mot de longueur fixe dans un texte

#### Plan du notebook:
1/ Implémentation d'un algorithme "Difference Cover size 3" pour construire une table de suffixe à partir d'un génome en un temps linéaire

### DC3
#### Première étape de DC3: division des positions de notre génome, tri des positions

In [132]:
import time

In [2]:
def asciiDC3 (seq) : 
    """
    Create sequence of ascii equivalent of each element of given list parameter.
    Add three sentinel numbers necessary for DC3 algorithm.
    
    Args:
        seq (list of str): list of elements to transform
    
    Return:
        (list of int)
    """
    asc=[]
    for i in seq : 
        asc.append(ord(i))
    
    return asc+[0,0,0]

##### Complexité:
Parcourir la séquence 1 fois implique, que asciiDC3 a une compléxité $O(n)$

In [3]:
def position1_2 (asc):
    """
    Create list of positions not multiple of 3
    
    Args:
        asc (list of int): sequence we want to extract a suffix array from
    
    Return:
         ind1+ind2 (list of int): list of positions not multiple of 3
    """
    ind1=[]
    ind2=[]
    for k in range(len(asc)-2):
        #attention on a peut etre fait de la merde ici, pas sure du -2
        if k%3==1 :
            ind1.append(k)
        if k%3==2:
            ind2.append(k)
    # dans le cas où T a un nombre d'élément multiple de 3, r12 ne contiendra pas le triplet [0,0,0].
    # Or, la fonction, "removesentinel" retire 1 élément notre suffix table, pour justement se débarrasser de [0,0,0]
    # J'ai donc décider de rajouter artificiellement [0,0,0] dans les cas où il n'est pas déjà là.
    if len(asc)%3==0:
        ind1.append(len(asc)-1-2)
        
    return ind1+ind2

##### Complexité:
Parcourir la séquence 1 fois implique, que position1_2 a une compléxité $O(n)$

In [134]:
def radix_with_p12(p,t):
    """
    Creates a list of couples containing:
    - sequences of three elements from "t" staring from positions in parameter "p"
    - the position p
    
    Args : 
        p (list of int): list of positions not multiple of 3 
        t (list of int): sequence we want to extract a suffix array from
    
    Return : 
        r (list of list): list of triplets and their starting positions
    
    """
    r=[]
    for i in range(len(p)):
        index=p[i]
        r.append([[t[index],t[index+1], t[index+2]],index])
    return r

##### Complexité:
Parcourir la séquence 1 fois implique, que radix_with_p12 a une compléxité $O(n)$

In [5]:
def sort_with_p12(array, alphabet, columnNumber):
        """
    Sorts parameter "array" using Radix Sort
    
    Args:
        array (list of list): list of triplets and their positions.
        
        alphabet (dictionnary): alphabet of our array. Ouput of function "alphabetT" or "alphabetR0_with_p12"
        
        columnNumber (int): number of last column of the lists in our paramater "array". 
                                    for r12, columnNumber=2
                                    for r0, columnNumber=1
    
    Return:
        array (list of list) : sorted list of triplets and their positions

    """
    if len(array) == 0:
        return array

  # Perform counting sort on each column, starting at the last

    column = columnNumber
    while column>=0: 
        array = countingSortByDigit_with_p12(array, alphabet, column)
        column-=1 #Switches column in array

    return array

def countingSortByDigit_with_p12(array, alphabet, column):
    """
    Sorts parameter "array" using Counting Sort
    
    Args:
        array (list of list): list of triplets and their positions.
        
        alphabet (dictionnary): alphabet of our array.O uput of function "alphabetT" or "alphabetR0_with_p12"
        
        columnNumber (int): number of last column of the lists in our paramater "array". 
                                    for r12, columnNumber=2
                                    for r0, columnNumber=1
    
    Return:
        array (list of list) : sorted list of triplets and their positions according to column number "column"

    """
    countIndex = -1
    count = [0] * len(alphabet)
    output = [None] * len(array)

  # Count frequencies
    for i in range(0, len(array)):
        countIndex = alphabet[array[i][0][column]]
        count[countIndex] += 1

  # Compute cumulates
    for i in range(1, len(alphabet)):
        count[i] += count[i - 1]

  # Move records
    for i in range(len(array) - 1, -1, -1):
        countIndex = alphabet[array[i][0][column]]
        count[countIndex] -= 1
        output[count[countIndex]] = array[i]
       
    return output

##### Complexité:
"sort" a la même complexité que "CountingSortByDigit", qui ,lui, a une complexité linéaire.

In [6]:
def triplets_are_equal(a,b):
    """
    Checks if a and b have the same elements.
    
    Args:
        a (list of int) 
        
        b (list of int)

    Return:
        (bool): True if the triplets are equal, False if not

    """
    for i in range(len(a)):
        if a[i]!=b[i]:
            return False
    return True


def ordre_with_p12(R12p12sorted, use_index_12=False):
    """
    Returns the order of each element of parameter "R12p12sorted".
    If use_index_12 is set to "True", it also returns a dictionnary, with the position of the triplet
    as a key, and its order as element.
    
    Example: 
        ordre_with_p12([ [ [0,0,0],1], [ [0,0,0],4], [ [0,0,1],7])=[1,1,2]
        
        ordre_with_p12([ [ [0,0,0],1], [ [0,0,0],4], [ [0,0,1],7], use_index_12=True)= [ [1,1,2], {1:1,4:1,7:2} ]
    
    args : 
        R12p12sorted (list) : list of triplets with their position in the sequence, ordered 
        
        use_index_12 (boolean) : True if you want a dictionnary back
        
    return : 
        if use_index_12=True : 
            order (list of int) :  order of each element of parameter "R12p12sorted"
            
            repetition (boolean) : If two triplets have the same ordre, "repetition"=True
            
            indexdict (dictionary) :keys are the positions p12 and elements are the order of the positions 
        
        if use_index_12=False : 
            order (list of int): order of each element of parameter "R12p12sorted"
    """
    # nous renovie la liste order du genre (1,2,2,3,4,4,5)  et un booléen indiquant s'il y a répétition
    index=1
    repetition=False
    order=[1]
    
    if use_index_12:
        indexdict={R12p12sorted[0][1]:1}
        
    for i in range(1, len(R12p12sorted)): 
        if triplets_are_equal(R12p12sorted[i-1][0], R12p12sorted[i][0]):
            order.append(index)
            repetition=True
        else : 
            index+=1
            order.append(index)
            
        if use_index_12:
            indexdict[R12p12sorted[i][1]]=order[i]
    if use_index_12:
        return order, repetition, indexdict
    else: 
        return order
            

##### Complexité:
triplets_are_equal a une complexité de $O(1)$ donc ordre_with_p12 a une complexité de $O(n)$ à cause la boucle for. 

In [7]:
def alphabetT(T):
    """
    Returns a dictionnary with the order of each "letter" constituting the parameter T.
    
    Example: 
        alphabetT([4,9,14,67])={4:0, 9:1, 14:2, 67:3}
    
    Args:
        T (list of int): the sequence we want a suffix array from
    
    Return:
        dic (dictionnary) : order of each "letter" constituting the parameter T

    """

    dic={}
    a=[]
    for i in range(len(T)):
        a.append(T[i])
    a.sort() 
    
    for element in a:
        if not (element in dic):
            dic[element]=len(dic)
    return dic

In [8]:
def alphabetR0_with_p12(R0p0):
    """
    Returns a dictionnary with the order of each "letter" constituting the parameter R0p0.
    Similar to the function alphabetT, coded specifically for the output of function position0_R0_p0
    
    Example: 
        alphabetR0_with_p12([[[65, 14], 0], [[67, 11], 3]]) = {11: 0, 14: 1, 65: 2, 67: 3}
    
    Args:
        R0p0 (list of list): list of couples and their position in the sequence we want a suffix array from.
                             output of function position0_R0_p0
                             [ [couple], position multiple of 3]
    
    Return:
        dic (dictionnary) : order of each "letter" constituting the parameter R0p0

    """
    
    dic={}
    a=[]
    for column in range(2):
        for i in range(len(R0p0)):
            a.append(R0p0[i][0][column])
    a.sort() 
    
    for element in a:
        if not (element in dic):
            dic[element]=len(dic)
    return dic

##### Complexité:
alphabetT a une complexité de $O(n)$ car la fonction parcourt toute la séquence une fois. Il en est de même pour alphabetR0_with_p12, car la seconde boucle for n'est que de taille 2. 

#### Deuxième étape de DC3: utilisation des premières positions triées pour construire second groupe de positions

In [137]:
def Tprime_with_p12(p12, indexdict):
    """
    Return a list of the order of each element of p12.
    The list will be the new sequence from which we want to extract a suffix array.
    
    Args : 
        p12 (list of int) : positions not multiple of 3
        
        indexdict (dictionnary) : keys are the positions p12 and elements are the order of the positions 
        
    Return :
        t(list of int): order of each element of p12
    """
    t=[]
    for p in p12:
        t.append(indexdict[p])
    return t

##### Complexité:
Parcourir la liste des positions p12 une fois implique que Tprime_with_p12 a une compléxité $O(n)$

In [10]:
def position0_R0_p0(T, index12dict):
    """
    Returns:
    -list of positions multiples of 3
    -list of lists, composed of: 1/ [element at position multiple of 3, order of the element at next position]
                                2/ position multiple of 3
    Args : 
        T (list of int) : sequence for which we are looking to extract the suffix array
        
        index12dict (dictionary) : dictionary which contains the positions of p12 as key and their order as elements
        
    Return :
        position (list): all positions multiple of 3
        
        R (list of list): lists each position with the elements at the next position

    """
    #attention T se finit avec trois 0
    #print("T")
    #print(T)
    position=[]
    R=[]
    for i in range(len(T)-3): #on ne prend pas en compte les trois 0 sentinelle
        if i%3==0:
            position.append(i)
            if i+1<len(T)-3:
                #print("i")
                #print(i)
                #print("T[i+1]")
                #print(T[i+1])
                R.append([[T[i],index12dict[i+1]],position[-1]])
            else:
                R.append([[T[i],1],position[-1]]) # car les sentinelles sont forcément les 1ères dans l'ordre alphabétique
    return position, R

##### Complexité:
Parcourir notre séquence initiale 1 fois implique que position0_R0_p0 a une compléxité $O(n)$. La tache "chercher le rang de la position suivante" a une complexité de $O(1)$ car on cherche une clé dans un dictionnaire

#### Troisième étape de DC3: combiner les positions triées pour trouver le suffix array.

In [11]:
def merge_with_p12(Tfinal, r0p0sorted, index12dict) :
        """
    Constructs a list of the positions in the right order to construct the suffix table of parameter "Tfinale"
    
    Args : 
        Tfinal(list of int) : sequence for which we want to create a suffix table 
        
        r0p0sorted(list of list) : sorted output of positionR0P0 
        
        index12dict(dictionary):  dictionary which contains the positions of p12 as key and their order as elements
    
    Return : 
        liste (list of int) : positions in the right order to construct the suffix table 
    """

    index12dictkeys=list(index12dict.keys())
    liste=[]
    A=0
    B=0
    while A<len(r0p0sorted) and B<len(index12dictkeys):
        a=r0p0sorted[A][1]
        b=index12dictkeys[B]
        if Tfinal[a]!=Tfinal[b] :
            minimum=min(Tfinal[a], Tfinal[b])
            
            if minimum == Tfinal[a]:
                A+=1
                liste.append(a)
            else: 
                B+=1
                liste.append(b)

        else :
            if b%3==1 : 
                longueur=len(liste)
                i=0
                if index12dict[a+1]<index12dict[b+1]:
                    liste.append(a)
                    A+=1
                else:
                    liste.append(b)
                
                    B+=1
                    
                    
            elif b%3==2 :
            
                if Tfinal[a+1]!=Tfinal[b+1] :
                   
                    minimum=min(Tfinal[a+1], Tfinal[b+1])
                    if minimum == Tfinal[a+1]:
                        A+=1
                    
                        liste.append(a)
                    else: 
                        B+=1
                        
                        liste.append(b)

                else:
                  
                    if index12dict[a+2]<index12dict[b+2]:
                        liste.append(a)
                        A+=1
                    else:
                        liste.append(b)
                        B+=1
                        

    if A==len(r0p0sorted):

        for i in range(B,len(index12dictkeys)):
            liste.append(index12dictkeys[i])
                
    if B==len(index12dictkeys):

        for i in range(A, len(r0p0sorted)):
            liste.append(r0p0sorted[i][1])

    return liste

##### Complexité:
La boucle while suivit de n'importe quelle des deux boucles for ne parcourt que n éléments. Les boucles ne sont pas imbriquées les unes dans les autres, et dans la boucle while, seules des opérations de complexité $O(1)$ prennent place.
Donc merge_with_p12 a une complexité de $O(n)$.

In [12]:
def merge_with_p12_debugg(Tfinal, r0p0sorted, index12dict) : 
    print("rappel des paramètres entrés pour la fonction merge")
    print("Tfinal")
    print(Tfinal)
    print("r0p0sorted")
    print(r0p0sorted)
    print("index12dict")
    print(index12dict)
    print("on construit une liste des clés du dictionnaire")
    print("qui est dans l'ordre normalement ahhh!")
    index12dictkeys=list(index12dict.keys())
    liste=[]
    A=0
    B=0
    while A<len(r0p0sorted) and B<len(index12dictkeys):
        print("rentre dans le while")
        a=r0p0sorted[A][1]
        b=index12dictkeys[B]
        if Tfinal[a]!=Tfinal[b] :
            minimum=min(Tfinal[a], Tfinal[b])
            
            if minimum == Tfinal[a]:
                print("a= "+str(a)+", b= "+str(b)+", on append le "+str(a))
                A+=1
                liste.append(a)
            else: 
                print("a= "+str(a)+", b= "+str(b)+", on append le "+str(b))
                B+=1
                liste.append(b)

        else :
            print("Tfinal[a]")
            print(Tfinal[a])
            print("Tfinal[b]")
            print(Tfinal[b])
            print("Tfinal[a]==Tfinal[b]")
            print(Tfinal[a]==Tfinal[b])
            if b%3==1 : 
                print(str(b)+" est congru à 1 modulo 3")
                longueur=len(liste)
                print("l'objet liste est pour l'instant constitué de: ")
                print(liste)
                i=0
                print("on compare les positions de "+str(a+1)+" et "+str(b+1))
                print("index12dict[" +str(a+1)+"]= ")
                print(index12dict[a+1])
                print("index12dict[" +str(b+1)+"]= ")
                print(index12dict[b+1])
                if index12dict[a+1]<index12dict[b+1]:
                    liste.append(a)
                    print("on append "+ str(a))
                    A+=1
                else:
                    liste.append(b)
                    print("on append "+ str(b))
                    B+=1
                    
                    
            elif b%3==2 :
                print(str(b)+"est congru à 2 modulo 3")
                if Tfinal[a+1]!=Tfinal[b+1] :
                    print("Tfinal[a+1]")
                    print(Tfinal[a+1])
                    print("Tfinal[b+1]")
                    print(Tfinal[b+1])
                    print("Tfinal[a+1]!=Tfinal[b+1]")
                    print(Tfinal[a+1]!=Tfinal[b+1])
                    minimum=min(Tfinal[a+1], Tfinal[b+1])
                    if minimum == Tfinal[a+1]:
                        A+=1
                        print("on append "+ str(a))
                        liste.append(a)
                    else: 
                        B+=1
                        print("on append "+ str(b))
                        liste.append(b)

                else:
                    print("Tfinal[a+1]")
                    print(Tfinal[a+1])
                    print("Tfinal[b+1]")
                    print(Tfinal[b+1])
                    print("Tfinal[a+1]==Tfinal[b+1]")
                    print(Tfinal[a+1]==Tfinal[b+1])
                    print("on compare a+2 et b+2")
                    if index12dict[a+2]<index12dict[b+2]:
                        liste.append(a)
                        A+=1
                    else:
                        liste.append(b)
                        B+=1
                        
    print("plus dans la boucle while")
    print("A "+str(A))
    print("B "+str(B))
    if A==len(r0p0sorted):
        print("on concatène les index12")
        for i in range(B,len(index12dictkeys)):
            liste.append(index12dictkeys[i])
                
    if B==len(index12dictkeys):
        print("on concatène les index0")
        for i in range(A, len(r0p0sorted)):
            liste.append(r0p0sorted[i][1])

    return liste

In [13]:
def removesentinel(index):
    """
    Return our suffix array without the position of the three sentinels numbers
    
    Args : 
        index (list of int) : list of indexes 
        
    Return :
        index[1:](list of int) : list of indexes without the sentinels
    
    """
    return index[1:]

##### Complexité
removesentinel a une complexité temporelle de $0(1)$.

In [14]:
def resumeHigherOrder_with_p12(index012prime, P12):
    """
    Returns a dictionnary with positions not multiple of 3 "P12" as key, and their order with 
    the positions of the suffix array "index012prime" as elements.
    
    Args : 
        index012prime (list of int) : suffix array of the positions not multiple of 3
        
        P12 (list of int) : positions not multiple of 3
    
    Return : 
        output (dictionnary) : positions not multiple of 3 "P12" as key, and their order with the positions of the suffix array "index012prime" as elements.
    """
    output={}
    for i in range(len(index012prime)):
        output[P12[index012prime[i]]]=i
        
    return output

### Testons le code avec l'exemple du cours

In [15]:
test_2='ATGCTAGCTGCCCTGATCTCTCTGA!'
p12_test_2=[1, 4, 7, 10, 13, 16, 19, 22, 25, 2, 5, 8, 11, 14, 17, 20, 23, 26]
r12_test_2=[[84, 71, 67], [84, 65, 71], [67, 84, 71], [67, 67, 67], [84, 71, 65], [84, 67, 84], [67, 84, 67], [84, 71, 65], [33, 0, 0], [71, 67, 84], [65, 71, 67], [84, 71, 67], [67, 67, 84], [71, 65, 84], [67, 84, 67], [84, 67, 84], [71, 65, 33], [0, 0, 0]]
alphabet_test_2={0: 0, 33: 1, 65: 2, 67: 3, 71: 4, 84: 5}


r12_p12_test_2=radix_with_p12(p12_test_2,asciiDC3(test_2))
print(r12_p12_test_2)



[[[84, 71, 67], 1], [[84, 65, 71], 4], [[67, 84, 71], 7], [[67, 67, 67], 10], [[84, 71, 65], 13], [[84, 67, 84], 16], [[67, 84, 67], 19], [[84, 71, 65], 22], [[33, 0, 0], 25], [[71, 67, 84], 2], [[65, 71, 67], 5], [[84, 71, 67], 8], [[67, 67, 84], 11], [[71, 65, 84], 14], [[67, 84, 67], 17], [[84, 67, 84], 20], [[71, 65, 33], 23], [[0, 0, 0], 26]]


In [16]:
r12_p12_sorted=sort_with_p12(r12_p12_test_2, alphabet_test_2, 2)
print(r12_p12_sorted)

[[[0, 0, 0], 26], [[33, 0, 0], 25], [[65, 71, 67], 5], [[67, 67, 67], 10], [[67, 67, 84], 11], [[67, 84, 67], 19], [[67, 84, 67], 17], [[67, 84, 71], 7], [[71, 65, 33], 23], [[71, 65, 84], 14], [[71, 67, 84], 2], [[84, 65, 71], 4], [[84, 67, 84], 16], [[84, 67, 84], 20], [[84, 71, 65], 13], [[84, 71, 65], 22], [[84, 71, 67], 1], [[84, 71, 67], 8]]


In [17]:
print(r12_p12_sorted[5][0])
print(r12_p12_sorted[6][0])
print(triplets_are_equal(r12_p12_sorted[5][0],r12_p12_sorted[6][0]))
ordre_test_2, repetition, indexdict_test_2=ordre_with_p12(r12_p12_sorted, True)
print(ordre_test_2)
print(indexdict_test_2)

[67, 84, 67]
[67, 84, 67]
True
[1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 12, 13, 13, 14, 14]
{26: 1, 25: 2, 5: 3, 10: 4, 11: 5, 19: 6, 17: 6, 7: 7, 23: 8, 14: 9, 2: 10, 4: 11, 16: 12, 20: 12, 13: 13, 22: 13, 1: 14, 8: 14}


In [18]:
p0, R0_p0_test_2=position0_R0_p0(asciiDC3(test_2), indexdict_test_2)

print(p0)
print(R0_p0_test_2)

alphabet_r0_test_2=alphabetR0_with_p12(R0_p0_test_2)
print(alphabet_r0_test_2)
    
r0p0sorted=sort_with_p12(R0_p0_test_2,alphabet_r0_test_2 ,1)
print(r0p0sorted)

order0=ordre_with_p12(r0p0sorted)
    

[0, 3, 6, 9, 12, 15, 18, 21, 24]
[[[65, 14], 0], [[67, 11], 3], [[71, 7], 6], [[71, 4], 9], [[67, 13], 12], [[65, 12], 15], [[84, 6], 18], [[67, 13], 21], [[65, 2], 24]]
{2: 0, 4: 1, 6: 2, 7: 3, 11: 4, 12: 5, 13: 6, 14: 7, 65: 8, 67: 9, 71: 10, 84: 11}
[[[65, 2], 24], [[65, 12], 15], [[65, 14], 0], [[67, 11], 3], [[67, 13], 12], [[67, 13], 21], [[71, 4], 9], [[71, 7], 6], [[84, 6], 18]]


### test zone avec abcabcabc

In [19]:
S="abcabcacab"
T=asciiDC3(S)
columnnumber=2
print(T)


[97, 98, 99, 97, 98, 99, 97, 99, 97, 98, 0, 0, 0]


In [20]:
p12=position1_2(T)
print(p12)
R12=radix_with_p12(p12, T)
print(R12)
alph=alphabetT(T)
alph
Rs12= sort_with_p12(R12, alph, columnnumber)
print(Rs12)

use_index12=True
order12, boolean, index12dict =ordre_with_p12(Rs12,use_index12)
print(order12)
print(boolean)
print(index12dict)

tprime=Tprime_with_p12(p12, index12dict)
print(tprime)

tprime=tprime+[0,0,0]
tprime

[1, 4, 7, 10, 2, 5, 8]
[[[98, 99, 97], 1], [[98, 99, 97], 4], [[99, 97, 98], 7], [[0, 0, 0], 10], [[99, 97, 98], 2], [[99, 97, 99], 5], [[97, 98, 0], 8]]
[[[0, 0, 0], 10], [[97, 98, 0], 8], [[98, 99, 97], 1], [[98, 99, 97], 4], [[99, 97, 98], 7], [[99, 97, 98], 2], [[99, 97, 99], 5]]
[1, 2, 3, 3, 4, 4, 5]
True
{10: 1, 8: 2, 1: 3, 4: 3, 7: 4, 2: 4, 5: 5}
[3, 3, 4, 1, 4, 5, 2]


[3, 3, 4, 1, 4, 5, 2, 0, 0, 0]

In [21]:
p12prime=position1_2(tprime)
print(p12prime)
R12prime=radix_with_p12(p12prime, tprime)
print(R12prime)
alphprime=alphabetT(tprime)
print(alphprime)
Rs12prime= sort_with_p12(R12prime, alphprime, columnnumber)
print(Rs12prime)

order12prime, boolean, index12primedict =ordre_with_p12(Rs12prime, True)
print(order12prime)
print(boolean)
print(index12primedict)

[1, 4, 7, 2, 5]
[[[3, 4, 1], 1], [[4, 5, 2], 4], [[0, 0, 0], 7], [[4, 1, 4], 2], [[5, 2, 0], 5]]
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
[[[0, 0, 0], 7], [[3, 4, 1], 1], [[4, 1, 4], 2], [[4, 5, 2], 4], [[5, 2, 0], 5]]
[1, 2, 3, 4, 5]
False
{7: 1, 1: 2, 2: 3, 4: 4, 5: 5}


In [22]:
p0prime, r0prime=position0_R0_p0(tprime, index12primedict)
print(p0prime)
print(r0prime)

[0, 3, 6]
[[[3, 2], 0], [[1, 4], 3], [[2, 1], 6]]


In [23]:
alph0prime=alphabetR0_with_p12(r0prime)
print(alph0prime)
r0sprime= sort_with_p12(r0prime, alph0prime, columnnumber-1)
print(r0sprime)

index120prime=merge_with_p12(tprime, r0sprime, index12primedict)
index120prime

{1: 0, 2: 1, 3: 2, 4: 3}
[[[1, 4], 3], [[2, 1], 6], [[3, 2], 0]]


[7, 3, 6, 0, 1, 2, 4, 5]

# Assemblage final

In [24]:
def almost_dc3_with_p12(T):
    # T est une liste de int

    
    columnNumber=2
    p12=position1_2(T)
    r12=radix_with_p12(p12,T)
    alphabet_T=alphabetT(T)
    r12sorted=sort_with_p12(r12, alphabet_T, columnNumber)    
    order12,repetition, index12dict=ordre_with_p12(r12sorted, True)
    if repetition:
        Tprim=Tprime_with_p12(p12, index12dict)+[0,0,0]
        index012=almost_dc3_with_p12(Tprim)
        index12dict=resumeHigherOrder_with_p12(index012, p12)
    p0,r0p0=position0_R0_p0(T, index12dict)
    alphabet_r0=alphabetR0_with_p12(r0p0)
    r0sorted=sort_with_p12(r0p0,alphabet_r0 ,columnNumber-1)
    index012=removesentinel(merge_with_p12(T, r0sorted, index12dict))
    
    return index012
   
    
    

In [25]:
def almost_dc3_with_p12_debugg(T):
    # T est une liste de int

    
    columnNumber=2
    p12=position1_2(T)
    
    print("p12")
    print(p12)
    print("taille de p12 avant récursion")
    print(len(p12))
    print("\n")
    
    r12=radix_with_p12(p12,T)
    print("r12")
    print(r12)
    print("\n")

    alphabet_T=alphabetT(T)
    print("alphabet de T")
    print(alphabet_T)
    
    r12sorted=sort_with_p12(r12, alphabet_T, columnNumber)
    print("r12sorted")
    print(r12sorted)
    print("\n")
    
    order12,repetition, index12dict=ordre_with_p12(r12sorted, True)
    print("order12")
    print(order12)
    print("index12dict")
    print(index12dict)
    print("entre-t-on dans la boucle de répétition?")
    print(repetition)
    print("\n")
    

    if repetition:

        
        Tprim=Tprime_with_p12(p12, index12dict)+[0,0,0]
        print("tprime")
        print(Tprim)
        print("taille de tprime")
        print(len(Tprim))
        print("\n")
        
        index012=almost_dc3_with_p12_debugg(Tprim)
        print("index012")
        print(index012)
        print("taille de index012")
        print(len(index012))
        print("\n")

        index12dict=resumeHigherOrder_with_p12(index012, p12)
        print("resume higher order index12dict")
        print(index12dict)
        print("taille de index12dict après retour à taille initiale")
        print(len(index12dict))
        print("\n")

    p0,r0p0=position0_R0_p0(T, index12dict)
    print("rappel de T")
    print(T)
    print("r0p0")
    print(r0p0)
    print("taille de r0p0")
    print(len(r0p0))
    print("\n")

    alphabet_r0=alphabetR0_with_p12(r0p0)
    print("alphabet_r0")
    print(alphabet_r0)
    print("\n")
    
    r0sorted=sort_with_p12(r0p0,alphabet_r0 ,columnNumber-1)
    print("r0sorted")
    print(r0sorted)
    print("taille de r0sorted")
    print(len(r0sorted))
    print("\n")

    #order0=ordre(r0sorted, index12)
    
    
    index012=removesentinel(merge_with_p12(T, r0sorted, index12dict))
    print("taille après la fonction removesentinel")
    print(len(index012))
    print("rappel de T")
    print(T)
    print("index012")
    print(index012)
    print("\n")
    
    for i in range(len(index012)):
        print(T[index012[i]:])
    
    return index012
   
    
    

## Test avec biopython 


In [26]:
pip install biopython

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
from Bio import SeqIO
genome=[]
nucleotide_genome=0
for seq_record in SeqIO.parse("/home/azarkua/Documents/2023-2024/omiques2/developement/omique2/genome.fna", "fasta"):
    genome.append(seq_record)
    nucleotide_genome+=len(seq_record.seq)
print(genome[13:])

[SeqRecord(seq=Seq('tgaaccctaaaccctaaaccctaaaccctaaacccctaaaccctaaaccctgaa...gtt'), id='NC_037283.1', name='NC_037283.1', description='NC_037283.1 Plasmodium falciparum 3D7 genome assembly, chromosome: 14', dbxrefs=[]), SeqRecord(seq=Seq('ATGATAAAATTTTTAAAACCTAAAATAAAAATATTAAAAAAATTAAATATACCT...ATT'), id='NC_036769.1', name='NC_036769.1', description='NC_036769.1 Plasmodium falciparum strain 3D7, apicoplast', dbxrefs=[])]


In [28]:
reads=[]
nucleotide_reads=0
for seq_record in SeqIO.parse("/home/azarkua/Documents/2023-2024/omiques2/developement/omique2/reads.fq", "fastq"):
    reads.append(seq_record)
    nucleotide_reads+=len(seq_record.seq)
    #if len(seq_record.seq)!= 100:
    #    print("False")
print(reads[:1])

[SeqRecord(seq=Seq('TTTCCTTTTTAAGCGTTTTATTTTTTAATAAAAAAAATATAGTATTATATAGTA...TAA'), id='NC_004325.2-100000', name='NC_004325.2-100000', description='NC_004325.2-100000', dbxrefs=[])]


In [29]:
temp=genome[1].seq.upper()
print(len(temp)//4)
temp[50:60]

236775


Seq('CCCTGAACCC')

Pour l'instant, notre code met environ 2min d'attente pour faire la suffix table de 25 000 nucléotides.

In [30]:
T=asciiDC3(temp)
#print(T)

In [31]:
suffix_array=almost_dc3_with_p12(T)
print(suffix_array[:100])
#print(timemeasuring)

[947101, 468510, 633566, 468511, 434614, 207415, 633567, 468512, 434615, 207416, 633568, 388233, 468513, 434616, 207417, 633569, 724610, 388234, 468514, 434617, 207418, 633570, 724611, 388235, 468515, 434618, 207419, 533168, 633571, 724612, 388236, 468516, 434619, 207420, 533169, 633572, 238221, 724613, 441106, 388237, 468517, 434620, 207421, 533170, 684120, 633573, 422507, 238222, 656252, 724614, 441107, 157882, 388238, 468518, 654531, 434621, 369055, 207422, 533171, 684121, 633574, 422508, 238223, 656253, 724615, 172540, 441108, 157883, 576520, 388239, 468519, 654532, 434622, 369056, 207423, 247439, 458613, 481497, 533172, 684122, 633575, 168552, 422509, 238224, 656254, 724616, 172541, 441109, 157884, 576521, 388240, 468520, 654533, 699572, 434623, 369057, 521750, 207424, 247440, 458614]


## Mesurer le temps que prend notre fonction

In [32]:
import pstats
import cProfile

In [33]:
cProfile.run("almost_dc3_with_p12(T)", "dc3_stats")
p = pstats.Stats("dc3_stats")
p.sort_stats("cumulative").print_stats()

Sun Nov  5 14:05:05 2023    dc3_stats

         28452198 function calls (28452193 primitive calls) in 24.924 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   24.924   24.924 {built-in method builtins.exec}
        1    0.429    0.429   24.923   24.923 <string>:1(<module>)
      6/1    1.153    0.192   24.495   24.495 /tmp/ipykernel_16511/3778850517.py:1(almost_dc3_with_p12)
       12    0.146    0.012    7.873    0.656 /tmp/ipykernel_16511/2101628537.py:1(sort_with_p12)
       30    7.728    0.258    7.728    0.258 /tmp/ipykernel_16511/2101628537.py:18(countingSortByDigit_with_p12)
        6    3.559    0.593    4.162    0.694 /tmp/ipykernel_16511/4110635339.py:1(merge_with_p12)
        6    3.533    0.589    3.616    0.603 /tmp/ipykernel_16511/2868372930.py:1(radix_with_p12)
        6    1.561    0.260    2.624    0.437 /tmp/ipykernel_16511/2442398083.py:8(ordre_with_p12)
        6    1.123  

### Idées d'optimisation  
-réussir à ne plus faire appel aux fonctions "alphabet"  

### BWT à l'aide de notre suffix table

In [62]:
def BWT_suffix_table(T,end_of_string=False):
    """
    Compute the BWT from the suffix table
    
    Args:
        T (str): string
        end_of_string (char): end of string character to append
    
    Return:
        bwt (str): BWT
    """
    if end_of_string==False:
        T += '!'
    suffix_array=almost_dc3_with_p12(asciiDC3(T)) 
    bwt = ""
    for i in suffix_array:
        bwt += T[i-1]
    return(bwt)

def BWT(T, suffix_table,end_of_string=False):
    """
    Compute the BWT from the suffix table
    
    Args:
        T (str): string
        end_of_string (char): end of string character to append
    
    Return:
        bwt (str): BWT
    """
    if end_of_string==False:
        T += '!'
    bwt = ""
    for i in suffix_table:
        bwt += T[i-1]
    return(bwt)

In [35]:
def run_length_encoding(S):
    """
    Encode sequence using the Run Length method
    
    Args:
        text (str): string to be shifted
    
    Return:
        str: run length
    """
    encoded_S= ""
    i=0
    number=1
    while i<len(S):
        encoded_S+=S[i]
        i+=1
        while i<len(S) and S[i-1]==S[i]:
            number+=1
            i+=1
        if number>1:
            encoded_S+=str(number)
        number=1
    return encoded_S

In [36]:
def print_suffix_table(sequence, sf, visualize_bwt=False):
    for i in range(len(sf)):
        if visualize_bwt:
            print(sequence[sf[i]-1:])
        else:
            print(sequence[sf[i]:])
    return

#### Testons notre fonction

In [37]:
test_2='ATGCTAGCTGCCCTGATCTCTCTGA!'
suffix_array_2_with_p12=almost_dc3_with_p12(asciiDC3(test_2)) 
print(suffix_array_2_with_p12)


[25, 24, 5, 15, 0, 10, 11, 3, 17, 19, 21, 12, 7, 23, 14, 9, 2, 6, 4, 16, 18, 20, 22, 13, 8, 1]


In [38]:
print_suffix_table(test_2,suffix_array_2_with_p12)

!
A!
AGCTGCCCTGATCTCTCTGA!
ATCTCTCTGA!
ATGCTAGCTGCCCTGATCTCTCTGA!
CCCTGATCTCTCTGA!
CCTGATCTCTCTGA!
CTAGCTGCCCTGATCTCTCTGA!
CTCTCTGA!
CTCTGA!
CTGA!
CTGATCTCTCTGA!
CTGCCCTGATCTCTCTGA!
GA!
GATCTCTCTGA!
GCCCTGATCTCTCTGA!
GCTAGCTGCCCTGATCTCTCTGA!
GCTGCCCTGATCTCTCTGA!
TAGCTGCCCTGATCTCTCTGA!
TCTCTCTGA!
TCTCTGA!
TCTGA!
TGA!
TGATCTCTCTGA!
TGCCCTGATCTCTCTGA!
TGCTAGCTGCCCTGATCTCTCTGA!


In [63]:
print(BWT_suffix_table(test_2, True))
print(BWT(test_2,suffix_array_2_with_p12, True))

AGTG!GCGTTTCGTTTTACACCCCCA
AGTG!GCGTTTCGTTTTACACCCCCA


In [40]:
print(run_length_encoding(BWT_suffix_table(test_2, True)))

AGTG!GCGT3CGT4ACAC5A


BWT sur notre génome

In [41]:
bwt_T=BWT_suffix_table(genome[1].seq.upper())
print(run_length_encoding(bwt_T)[:100])

ACT2AT2A5TA4CA10GA8TACA5CATACA2TA2TAGA8CA2CA6G2CA3GA11CA2TA12TA3TA4TA4TA5CA13CA2TA3CA3TA7CA3TA6TGTA2


Mesurons le temps que l'opération prend

In [42]:
cProfile.run("run_length_encoding(BWT_suffix_table(genome[1].seq.upper()))", "bwt_stats")
p = pstats.Stats("bwt_stats")
p.sort_stats("cumulative").print_stats()

Sun Nov  5 14:05:56 2023    bwt_stats

         38466892 function calls (38466887 primitive calls) in 28.881 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   28.881   28.881 {built-in method builtins.exec}
        1    0.034    0.034   28.881   28.881 <string>:1(<module>)
        1    0.742    0.742   28.380   28.380 /tmp/ipykernel_16511/2127043640.py:1(BWT_suffix_table)
      6/1    1.112    0.185   25.994   25.994 /tmp/ipykernel_16511/3778850517.py:1(almost_dc3_with_p12)
       12    0.140    0.012    7.423    0.619 /tmp/ipykernel_16511/2101628537.py:1(sort_with_p12)
       30    7.283    0.243    7.283    0.243 /tmp/ipykernel_16511/2101628537.py:18(countingSortByDigit_with_p12)
        6    3.533    0.589    4.143    0.691 /tmp/ipykernel_16511/4110635339.py:1(merge_with_p12)
        6    3.372    0.562    3.510    0.585 /tmp/ipykernel_16511/3786488779.py:1(position0_R0_p0)
        6    3.28

## String search with BWT

version du prof

In [43]:
from collections import Counter

In [64]:
def occurrence_indexer(S):
    K = []
    last_index = {}
    for s in S:
        if s not in last_index: 
            last_index[s] = 0
        K.append(last_index[s])
        last_index[s] += 1
    return(K)

def lettre_et_occurence(BWT,pattern_letter, position_in_BWT, occurence_index):
    output=[]
    for i in range(len(position_in_BWT)):
        if BWT[position_in_BWT[i]]==pattern_letter:
            output.append([BWT[position_in_BWT[i]],occurence_index[position_in_BWT[i]]])
    return output

def new_suffix_table_positions(BWT,letter_and_occurence, counts):
    # Counter({'T': 3, 'A': 2, 'B': 2, 'C': 2, '!': 1})
    output=[]
    for i in range(len(letter_and_occurence)):
        letter=letter_and_occurence[i][0]
        occurence=letter_and_occurence[i][1]
        new_position_to_evaluate=occurence + sum([counts[char] for char in counts if char < letter])
        output.append(new_position_to_evaluate)
    return output

def initialize_suffix_table_positions(counts, pattern_letter):
    start=sum([counts[char] for char in counts if char < pattern_letter])
    end=start+counts[pattern_letter]
    return [i for i in range(start, end)]
    
def find_patterns_in_sequence_with_dc3(sequence, pattern, sequence_has_end_character=False):
    output=[]
    
    if sequence_has_end_character==False:
        sequence += '!'
        
    sf=almost_dc3_with_p12(asciiDC3(sequence)) 
    BWT = ""
    for i in sf:
        BWT += sequence[i-1]
    
    
    
    #print("BWT")
    #print(BWT)
    #print("\n")
    occurence_index = occurrence_indexer(BWT)
    #print('occurence_index')
    #print(occurence_index)
    #print("\n")
    counts = Counter(BWT)
    #print("counts")
    #print(counts)
    #print("\n")
    index=len(pattern)-1
    pattern_letter=pattern[index]
    
    suffix_table_positions=initialize_suffix_table_positions(counts, pattern_letter)
    #print("suffix_table_positions")
    #print(suffix_table_positions)
    #print("\n")
    while index>0 and len(suffix_table_positions)>0:
        
        index-=1
        #print("index")
        #print(index)
        #print("\n")
        pattern_letter=pattern[index]
        #print("pattern_letter")
        #print(pattern_letter)
        #print("\n")
        
        A=lettre_et_occurence(BWT,pattern_letter,suffix_table_positions, occurence_index)
        #print("A")
        #print(A)
        #print("\n")
        
        suffix_table_positions=new_suffix_table_positions(BWT, A, counts)
        #print("suffix_table_positions")
        #print(suffix_table_positions)
        #print("\n")
        
        
    for i in range(len(suffix_table_positions)):
        output.append(sf[suffix_table_positions[i]])
        

    return output

def find_patterns_in_sequence(sf, BWT, pattern):
    output=[]

    
    
    
    #print("BWT")
    #print(BWT)
    #print("\n")
    occurence_index = occurrence_indexer(BWT)
    #print('occurence_index')
    #print(occurence_index)
    #print("\n")
    counts = Counter(BWT)
    #print("counts")
    #print(counts)
    #print("\n")
    index=len(pattern)-1
    pattern_letter=pattern[index]
    
    suffix_table_positions=initialize_suffix_table_positions(counts, pattern_letter)
    #print("suffix_table_positions")
    #print(suffix_table_positions)
    #print("\n")
    while index>0 and len(suffix_table_positions)>0:
        
        index-=1
        #print("index")
        #print(index)
        #print("\n")
        pattern_letter=pattern[index]
        #print("pattern_letter")
        #print(pattern_letter)
        #print("\n")
        
        A=lettre_et_occurence(BWT,pattern_letter,suffix_table_positions, occurence_index)
        #print("A")
        #print(A)
        #print("\n")
        
        suffix_table_positions=new_suffix_table_positions(BWT, A, counts)
        #print("suffix_table_positions")
        #print(suffix_table_positions)
        #print("\n")
        
        
    for i in range(len(suffix_table_positions)):
        output.append(sf[suffix_table_positions[i]])
        

    return output
        
        
    

In [91]:
def new_suffix_table_positions_faster(BWT,pattern_letter, position_in_BWT, occurence_index, counts):
    # Counter({'T': 3, 'A': 2, 'B': 2, 'C': 2, '!': 1})
    output=[]
    letter_and_occurence=[]
    for i in range(len(position_in_BWT)):
        if BWT[position_in_BWT[i]]==pattern_letter:
            letter_and_occurence=[BWT[position_in_BWT[i]],occurence_index[position_in_BWT[i]]]
            letter=letter_and_occurence[0]
            occurence=letter_and_occurence[1]
            new_position_to_evaluate=occurence + sum([counts[char] for char in counts if char < letter])
            output.append(new_position_to_evaluate)
    return output

def find_patterns_in_sequence_faster(sf, BWT, pattern):
    output=[]
    occurence_index = occurrence_indexer(BWT)
    counts = Counter(BWT)
    index=len(pattern)-1
    pattern_letter=pattern[index]
    
    suffix_table_positions=initialize_suffix_table_positions(counts, pattern_letter)
    while index>0 and len(suffix_table_positions)>0:
        index-=1
        pattern_letter=pattern[index]
        suffix_table_positions=new_suffix_table_positions_faster(BWT,pattern_letter,suffix_table_positions,occurence_index,counts) 
        
    for i in range(len(suffix_table_positions)):
        output.append(sf[suffix_table_positions[i]])
        
    return output

#### Testons notre fonction

In [92]:
test_mltpl='ABCDEFGHIJKLMNOPPPPPPABCDE!'
sf=almost_dc3_with_p12(asciiDC3(test_mltpl))
print(sf)

[26, 21, 0, 22, 1, 23, 2, 24, 3, 25, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 20, 19, 18, 17, 16, 15]


In [96]:
positions_of_pattern=find_patterns_in_sequence_with_dc3(test_mltpl, 'GHIJK', sequence_has_end_character=True)
print(positions_of_pattern)
for p in positions_of_pattern:
    print(test_mltpl[p:p+6])


[6]
GHIJKL


In [47]:
#print_suffix_table(test_mltpl, sf, visualize_bwt=True)

### Testons avec notre génome

In [97]:
sf=almost_dc3_with_p12(asciiDC3(genome[0].seq.upper()))
bwt=BWT(genome[0].seq.upper(), sf, False)

In [98]:
pattern=genome[0].seq.upper()[:10]
pattern

Seq('TGAACCCTAA')

In [101]:
positions_of_pattern=find_patterns_in_sequence_faster(sf, bwt, pattern)
print(sorted(positions_of_pattern))
#for p in positions_of_pattern:
    #print(genome[1].seq.upper()[p:p+len(pattern)])

[0, 35, 49, 70, 91, 119, 140, 174, 195, 209, 230, 244]


In [51]:
cProfile.run("find_patterns_in_sequence(genome[1].seq.upper(), pattern, sequence_has_end_character=False)", "pattern_matching_stats")
p = pstats.Stats("pattern_matching_stats")
p.sort_stats("cumulative").print_stats()

Sun Nov  5 14:06:45 2023    pattern_matching_stats

         38139081 function calls (38139075 primitive calls) in 29.532 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   29.532   29.532 {built-in method builtins.exec}
        1    0.044    0.044   29.532   29.532 <string>:1(<module>)
        1    0.755    0.755   29.487   29.487 /tmp/ipykernel_16511/231063892.py:33(find_patterns_in_sequence)
      6/1    1.169    0.195   26.627   26.627 /tmp/ipykernel_16511/3778850517.py:1(almost_dc3_with_p12)
       12    0.143    0.012    7.703    0.642 /tmp/ipykernel_16511/2101628537.py:1(sort_with_p12)
       30    7.560    0.252    7.560    0.252 /tmp/ipykernel_16511/2101628537.py:18(countingSortByDigit_with_p12)
        6    5.576    0.929    5.672    0.945 /tmp/ipykernel_16511/2868372930.py:1(radix_with_p12)
        6    3.599    0.600    4.212    0.702 /tmp/ipykernel_16511/4110635339.py:1(merge_with_p

### Travaillons avec des kmers

In [88]:
def assembler_des_kmers(numero_du_kmer, position_ds_genome,longueur, liste_position_kmers):
    compteur_de_kmer=1 #compte le nb de kmers qu'on arrive à aligner
    last_position=0
    i=numero_du_kmer+1
    difference=1
    
    while i<len(liste_position_kmers):
        j=0
        previous_compteur=compteur_de_kmer
        k=0
        while j<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:
            if position_ds_genome+difference==liste_position_kmers[i][j]:
                compteur_de_kmer+=1
                last_position=liste_position_kmers[i][j]
            j+=1
            
        if compteur_de_kmer==previous_compteur:
            i+=longueur
            difference+=longueur
            if  i>= len(liste_position_kmers):
                return [[position_ds_genome,last_position+longueur],compteur_de_kmer]
            else:
        
                ### cas d'une mutation
                k=0

                while k<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:
                    if position_ds_genome+difference==liste_position_kmers[i][k]:
                        compteur_de_kmer+=1
                        last_position=liste_position_kmers[i][k]
                    k+=1



                if compteur_de_kmer==previous_compteur:
                    ###cas d'une addition
                    difference-=1
                    l=0
                    while l<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:

                        if position_ds_genome+difference==liste_position_kmers[i][l]:
                            compteur_de_kmer+=1
                            last_position=liste_position_kmers[i][l]
                        l+=1

                    if compteur_de_kmer==previous_compteur:
                        ###cas d'une déletions
                        difference+=1
                        i-=1
                        m=0
                        while m<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:

                            if position_ds_genome+difference==liste_position_kmers[i][m]:
                                compteur_de_kmer+=1
                                last_position=liste_position_kmers[i][m]
                            m+=1

                        if compteur_de_kmer==previous_compteur:
                            ### cas où l'alignement est terminé
                            return [[position_ds_genome,last_position+longueur],compteur_de_kmer]

        i+=1
        difference+=1   
        
    return [[position_ds_genome,last_position+longueur],compteur_de_kmer]

In [52]:
def assembler_des_kmers_debugg(numero_du_kmer, position_ds_genome,longueur, liste_position_kmers):
    compteur_de_kmer=1 #compte le nb de kmers qu'on arrive à aligner
    last_position=0
    i=numero_du_kmer+1
    difference=1
    
    while i<len(liste_position_kmers):
        #print("i au début du while")
        #print(i)
        #print("\n")
        j=0
        previous_compteur=compteur_de_kmer
        #print("le compteur est à")
        #print(compteur_de_kmer)
        #print("\n")
        #print("on s'interesse au kmer numero")
        #print(i)
        #print("qui est")
        #print(liste_position_kmers[i])
        k=0
        while j<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:
            #print("on cherche la position")
            #print(position_ds_genome+difference)
            if position_ds_genome+difference==liste_position_kmers[i][j]:
                compteur_de_kmer+=1
                last_position=liste_position_kmers[i][j]
            j+=1
            
        if compteur_de_kmer==previous_compteur:
            i+=longueur
            difference+=longueur
            if  i>= len(liste_position_kmers):
                return [[position_ds_genome,last_position+longueur],compteur_de_kmer]
            else:
        
                ### cas d'une mutation
                #print("cas d'une mutation")
                #print("on s'interesse au kmer numero")
                #print(i)
                #print("qui est")
                #print(liste_position_kmers[i])
                k=0

                while k<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:
                    #print("on cherche la position")
                    #print(position_ds_genome+difference)
                    if position_ds_genome+difference==liste_position_kmers[i][k]:
                        compteur_de_kmer+=1
                        last_position=liste_position_kmers[i][k]
                    k+=1



                if compteur_de_kmer==previous_compteur:
                    ###cas d'une addition
                    #print("cas d'une addition")
                    difference-=1
                    l=0
                    while l<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:

                        if position_ds_genome+difference==liste_position_kmers[i][l]:
                            compteur_de_kmer+=1
                            last_position=liste_position_kmers[i][l]
                        l+=1

                    if compteur_de_kmer==previous_compteur:
                        ###cas d'une déletions
                        #print("cas d'une délétion")
                        difference+=1
                        i-=1
                        m=0
                        while m<len(liste_position_kmers[i]) and compteur_de_kmer==previous_compteur:

                            if position_ds_genome+difference==liste_position_kmers[i][m]:
                                compteur_de_kmer+=1
                                last_position=liste_position_kmers[i][m]
                            m+=1

                        if compteur_de_kmer==previous_compteur:
                            ### cas où l'alignement est terminé
                            return [[position_ds_genome,last_position+longueur],compteur_de_kmer]

        i+=1
        difference+=1   
        
    return [[position_ds_genome,last_position+longueur],compteur_de_kmer]

In [53]:
def kmer(sequence, longueur):
    n = len(sequence)
    kmer=[]
    for i in range(n-longueur+1):
        kmer.append(sequence[i:i+longueur])
    return kmer
    

In [54]:
read="ABCDEFGHIJKLMNOPQRSZUV"
print(len(read))
print(kmer(read,5))

22
['ABCDE', 'BCDEF', 'CDEFG', 'DEFGH', 'EFGHI', 'FGHIJ', 'GHIJK', 'HIJKL', 'IJKLM', 'JKLMN', 'KLMNO', 'LMNOP', 'MNOPQ', 'NOPQR', 'OPQRS', 'PQRSZ', 'QRSZU', 'RSZUV']


In [55]:
genom="ABCDEFGHIJKLMNOPQRSTUVWPPPPPABCDEFGHWWWWWWWWWWNOPQRSTUV!"
liste_kmer=kmer(read, 5)
print(liste_kmer)
print(len(liste_kmer))

['ABCDE', 'BCDEF', 'CDEFG', 'DEFGH', 'EFGHI', 'FGHIJ', 'GHIJK', 'HIJKL', 'IJKLM', 'JKLMN', 'KLMNO', 'LMNOP', 'MNOPQ', 'NOPQR', 'OPQRS', 'PQRSZ', 'QRSZU', 'RSZUV']
18


In [57]:
positions=[]
for i in range(len(liste_kmer)):
    positions.append(find_patterns_in_sequence(genom, liste_kmer[i],sequence_has_end_character=True))
print(positions)
print(positions[5][0])

[[0, 28], [1, 29], [2, 30], [3, 31], [4], [5], [6], [7], [8], [9], [10], [11], [12], [46, 13], [47, 14], [], [], []]
5


In [58]:
assembler_des_kmers(0, positions[0][0],5,positions)

[[0, 19], 15]

In [59]:
def alignement_maximum_de_kmer(liste_position_kmers, longueur):
    output=[[0,0],0]
    maximum=0
    for i in range(longueur+1):
        current_kmer=liste_position_kmers[i]
        for p in current_kmer:
            alignement=assembler_des_kmers(i, p, longueur, liste_position_kmers)
            if maximum<alignement[1]:
                output=alignement
                maximum=alignement[1]
    return output

In [85]:
def complementaireInverse(sequence):
    ci = []
    for i in range(len(sequence) - 1, -1, -1):
        if sequence[i] == 'A':
            ci.append('T')
        elif sequence[i] == 'T':
            ci.append('A')
        elif sequence[i] == 'C':
            ci.append('G')
        elif sequence[i] == 'G':
            ci.append('C')
    return ''.join(ci)

In [60]:
alignement_maximum_de_kmer(positions, 5)

[[0, 19], 15]

In [75]:
genome[0].seq.upper()
reads[1].seq.upper()
liste_de_kmer=kmer(reads[2].seq.upper(), 10)
position=[]
for i in range(len(liste_de_kmer)):
    position.append(find_patterns_in_sequence(sf,bwt, liste_de_kmer[i]))


[[471737, 434515], [203927], [398898, 471739, 4573], [398899, 113502, 471740, 391868], [325748, 398900, 113503, 471741, 182775], [68252, 450995, 113504, 487338, 416677, 227345, 343947, 262351, 471742, 264435, 542720, 423234, 244111, 508193, 68575, 328524, 215081, 200788, 299644, 440172, 599664, 182776, 272603, 598101], [96497, 37693, 80472, 77000, 390523, 68253, 450996, 389536, 113505, 487339, 7024, 288851, 236058, 189043, 288694, 329747, 357594, 564712, 416678, 227346, 383355, 84109, 433090, 433410, 151985, 343948, 286795, 262352, 471743, 486290, 264436, 566513], [236059, 70816, 188659, 494667, 189044, 571421, 331913, 288695, 310775, 177481, 334350, 329748, 357595, 564713, 562511, 177608, 416679, 418328, 227347, 182289, 345619, 131471, 334721, 262540, 92399, 206037, 206103, 383356, 91218, 50542, 59992, 84110, 327935, 287246, 65444, 433091, 125007, 433411, 107074, 174821, 151986, 178612, 463003, 374186, 567488, 154790, 72766, 343949, 615340, 286796, 89101, 262353, 471744, 486291, 33307

In [76]:
alignement_maximum_de_kmer(position, 10)

[[471739, 471836], 58]

In [102]:
def align_read(chromosome, reads_list, longueur_kmer):
    sf=almost_dc3_with_p12(asciiDC3(chromosome))
    bwt=BWT(chromosome, sf)
    output=[]
    
    for i in range(len(reads_list)):
        liste_de_kmer=kmer(reads_list[i].seq.upper(), longueur_kmer)
        position=[]
        for j in range(len(liste_de_kmer)):
            position.append(find_patterns_in_sequence_faster(sf,bwt, liste_de_kmer[j]))
        
        alignment=alignement_maximum_de_kmer(position, longueur_kmer)
        
        if (alignment[0][1]-alignment[0][0])<30:
            # on cherche à aligner le complémentaire inverse
            liste_de_kmer=kmer(complementaireInverse(reads_list[i].seq.upper()), longueur_kmer)
            position=[]
            for j in range(len(liste_de_kmer)):
                position.append(find_patterns_in_sequence_faster(sf,bwt, liste_de_kmer[j]))
                
            alignment=alignement_maximum_de_kmer(position, longueur_kmer)
                
        output.append({"read":i, "alignement":alignment})
    return output
        

In [105]:
start=time.time()
align=align_read(genome[0].seq.upper(),reads[:10], 10)
end=time.time()
print(align)
print(end-start)

[{'read': 0, 'alignement': [[131734, 131771], 28]}, {'read': 1, 'alignement': [[143900, 143942], 13]}, {'read': 2, 'alignement': [[471739, 471836], 58]}, {'read': 3, 'alignement': [[65426, 65521], 76]}, {'read': 4, 'alignement': [[152683, 152749], 27]}, {'read': 5, 'alignement': [[433417, 433436], 10]}, {'read': 6, 'alignement': [[592083, 592152], 40]}, {'read': 7, 'alignement': [[270169, 270269], 61]}, {'read': 8, 'alignement': [[179072, 179145], 54]}, {'read': 9, 'alignement': [[463570, 463670], 61]}]
510.52226877212524


In [119]:
def check_for_mutation(index, read_decalage,genome_decalage, read,genome):
    return read[read_decalage+index+1]==genome[genome_decalage+index+1]


def check_for_deletion(index, read_decalage, genome_decalage, read,genome):
    return read[read_decalage+index]==genome[genome_decalage+index+1]

def check_for_addition(index, read_decalage, genome_decalage, read,genome):
    return read[read_decalage+index+1]==genome[genome_decalage+index]

def is_my_read_at_position(position_in_genome, read,genome):
    read_decalage=0
    genome_decalage=position_in_genome
    i=0
    while i+read_decalage<len(read):
        if read[read_decalage+i]!=genome[genome_decalage+i]:
            if check_for_mutation(i, read_decalage,genome_decalage, read,genome):
                print("Le read contient une mutation à la position "+str(i))
                print(read[read_decalage+i-5:read_decalage+i+5])
                print(genome[genome_decalage+i-5:genome_decalage+i+5])
                i+=1
                
            elif check_for_deletion(i, read_decalage,genome_decalage, read,genome):
                print("Le read contient une déletion à la position "+str(i))
                print(read[read_decalage+i-5:read_decalage+i+5])
                print(genome[genome_decalage+i-5:genome_decalage+i+5])
                genome_decalage+=1
                i+=1
                
            elif check_for_addition(i, read_decalage, genome_decalage, read,genome):
                print("Le read contient une addition à la position "+str(i))
                print(read[read_decalage+i-5:read_decalage+i+5])
                print(genome[genome_decalage+i-5:genome_decalage+i+5])
                i+=1
                read_decalage+=1
                
            else:
                i+=1
                print("...")
        else:
            i+=1
    return

In [131]:
is_my_read_at_position(471737, reads[2].seq.upper(),genome[0].seq.upper())

In [130]:
print(reads[2].seq.upper())
print(genome[0].seq.upper()[471737:471839])

TCATGATTTACATATATTTGTAAAACATATATAATCTGTCCAGACATATTATATAATTGATAATATAATATATATATATATATATAAATTATTACTTCTC
TCATGATTTACATATATTTGTAAAACATATATAATCTGTCCAGACATATTATATAATTGATAATATAATATATATATATATATATAAATTATTACTTCTCCA


In [104]:
cProfile.run("align_read(genome[0].seq.upper(),reads[:2], 10)", "pattern_matching_stats")
p = pstats.Stats("pattern_matching_stats")
p.sort_stats("cumulative").print_stats()

Sun Nov  5 15:16:58 2023    pattern_matching_stats

         297656368 function calls (297656177 primitive calls) in 121.560 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  121.560  121.560 {built-in method builtins.exec}
        1    0.019    0.019  121.560  121.560 <string>:1(<module>)
        1    2.186    2.186  121.541  121.541 /tmp/ipykernel_16511/2110851397.py:1(align_read)
      273    0.673    0.002  101.218    0.371 /tmp/ipykernel_16511/4286594490.py:14(find_patterns_in_sequence_faster)
      273   38.552    0.141   46.724    0.171 /tmp/ipykernel_16511/688712901.py:1(occurrence_indexer)
     2442   28.665    0.012   46.269    0.019 /tmp/ipykernel_16511/4286594490.py:1(new_suffix_table_positions_faster)
      7/1    0.689    0.098   16.738   16.738 /tmp/ipykernel_16511/3778850517.py:1(almost_dc3_with_p12)
 32289828   12.890    0.000   12.890    0.000 /tmp/ipykernel_16511/4286594490.py